# Lab 3: River Profiles and Drainage Metrics

**Landscape and Surface Processes Dynamics**

---

## Objectives
- Learn how to extract and analyze drainage networks from Digital Elevation Models (DEMs)
- Practice slope–area analysis for understanding landscape evolution
- Extract and analyze longitudinal river profiles
- Identify knickpoints and understand their tectonic and climatic significance
- Compare concavity indices between tectonically active and stable basins

---

## Prerequisites
This notebook uses Python libraries for geospatial analysis. We'll work with:
- **richdem**: DEM analysis and hydrological processing
- **numpy/scipy**: Numerical computations
- **matplotlib**: Visualization
- **rasterio**: Raster data handling

---

## Setup and Installation

First, let's install the required packages:

In [ ]:
# Install required packages
!pip install richdem rasterio matplotlib numpy scipy elevation

# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import richdem as rd
from scipy import ndimage
import rasterio
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

## Data Acquisition

For this lab, you can either:
1. Upload your own DEM file
2. Use sample DEM data
3. Download DEM from online sources (e.g., SRTM, ASTER)

We'll demonstrate with a sample dataset approach:

In [ ]:
# Option 1: Upload your DEM file
# from google.colab import files
# uploaded = files.upload()
# dem_file = list(uploaded.keys())[0]

# Option 2: Create synthetic DEM for demonstration
# We'll create a simple synthetic landscape for demonstration
def create_synthetic_dem(size=500, seed=42):
    """Create a synthetic DEM with realistic features"""
    np.random.seed(seed)
    
    # Create base topography with gradient
    x = np.linspace(0, 1, size)
    y = np.linspace(0, 1, size)
    X, Y = np.meshgrid(x, y)
    
    # Base elevation: higher in upper right, lower in lower left
    base = 1000 * (0.7 * X + 0.3 * Y) + 500
    
    # Add some random noise for realism
    noise = np.random.randn(size, size) * 10
    
    # Add some valley features
    for i in range(3):
        center_x = np.random.uniform(0.2, 0.8)
        center_y = np.random.uniform(0.2, 0.8)
        valley = -100 * np.exp(-((X - center_x)**2 + (Y - center_y)**2) / 0.05)
        base += valley
    
    dem = base + noise
    return dem

# Create synthetic DEM
dem_array = create_synthetic_dem()

# Convert to richdem format
dem = rd.rdarray(dem_array, no_data=-9999)

print(f"DEM loaded successfully!")
print(f"DEM shape: {dem.shape}")
print(f"Elevation range: {np.min(dem):.2f} to {np.max(dem):.2f} m")

In [ ]:
# Visualize the DEM
plt.figure(figsize=(12, 10))
plt.imshow(dem, cmap='terrain', aspect='auto')
plt.colorbar(label='Elevation (m)', shrink=0.8)
plt.title('Digital Elevation Model (DEM)', fontsize=14, fontweight='bold')
plt.xlabel('X (pixels)')
plt.ylabel('Y (pixels)')
plt.tight_layout()
plt.show()

## Step 1: DEM Preprocessing - Pit Filling

**Theory:**
DEMs often contain "pits" or "sinks" - local depressions that prevent water from flowing continuously downslope. These can be:
- Artifacts from data acquisition/processing
- Real features (lakes, closed depressions)

**Pit filling** removes these depressions to ensure continuous flow routing, which is essential for drainage network extraction.

**Algorithm:** We'll use the priority-flood algorithm (Barnes et al., 2014), which efficiently fills depressions while preserving topographic details.

In [ ]:
# Create a copy for comparison
dem_original = dem.copy()

# Fill depressions using priority-flood algorithm
print("Filling depressions...")
dem_filled = rd.FillDepressions(dem, epsilon=True, in_place=False)
print("Depression filling complete!")

# Calculate the difference to see what was filled
fill_depth = dem_filled - dem_original

print(f"Maximum fill depth: {np.max(fill_depth):.2f} m")
print(f"Pixels modified: {np.sum(fill_depth > 0)} ({100*np.sum(fill_depth > 0)/dem.size:.2f}%)")

In [ ]:
# Visualize pit filling results
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Original DEM
im1 = axes[0].imshow(dem_original, cmap='terrain', aspect='auto')
axes[0].set_title('Original DEM', fontsize=12, fontweight='bold')
axes[0].set_xlabel('X (pixels)')
axes[0].set_ylabel('Y (pixels)')
plt.colorbar(im1, ax=axes[0], label='Elevation (m)', shrink=0.8)

# Filled DEM
im2 = axes[1].imshow(dem_filled, cmap='terrain', aspect='auto')
axes[1].set_title('Filled DEM', fontsize=12, fontweight='bold')
axes[1].set_xlabel('X (pixels)')
axes[1].set_ylabel('Y (pixels)')
plt.colorbar(im2, ax=axes[1], label='Elevation (m)', shrink=0.8)

# Fill depth
im3 = axes[2].imshow(fill_depth, cmap='YlOrRd', aspect='auto')
axes[2].set_title('Fill Depth', fontsize=12, fontweight='bold')
axes[2].set_xlabel('X (pixels)')
axes[2].set_ylabel('Y (pixels)')
plt.colorbar(im3, ax=axes[2], label='Depth (m)', shrink=0.8)

plt.tight_layout()
plt.show()

## Step 2: Extract Drainage Network with Threshold Contributing Area

**Theory:**
Drainage networks form where sufficient water accumulates to initiate channels. We extract the network by:
1. Computing **flow direction** (D8 or D-infinity algorithm)
2. Calculating **flow accumulation** (upstream contributing area)
3. Applying a **threshold** to define channel initiation

**Threshold selection:**
- Too low: Dense network with many spurious channels
- Too high: Misses smaller tributaries
- Typical: 0.1-1 km² depending on climate and lithology

In [ ]:
# Calculate flow accumulation
print("Computing flow accumulation...")
accum = rd.FlowAccumulation(dem_filled, method='D8')
print("Flow accumulation complete!")

# Assuming 30m pixel resolution (typical for SRTM)
pixel_area_km2 = (30 * 30) / 1e6  # Convert m² to km²
accum_area = accum * pixel_area_km2

print(f"Maximum contributing area: {np.max(accum_area):.2f} km²")
print(f"Flow accumulation computed for {dem.size} pixels")

In [ ]:
# Define threshold for channel initiation (in km²)
threshold_area_km2 = 0.5  # Adjust based on your study area

# Extract drainage network
channel_network = accum_area > threshold_area_km2

print(f"Channel network extracted with threshold: {threshold_area_km2} km²")
print(f"Number of channel pixels: {np.sum(channel_network)}")
print(f"Drainage density: {np.sum(channel_network) * 30 / (dem.size * 900):.3f} km/km²")

In [ ]:
# Visualize drainage network
fig, axes = plt.subplots(1, 2, figsize=(16, 7))

# Flow accumulation (log scale for better visualization)
im1 = axes[0].imshow(np.log10(accum + 1), cmap='Blues', aspect='auto')
axes[0].set_title('Flow Accumulation (log₁₀)', fontsize=12, fontweight='bold')
axes[0].set_xlabel('X (pixels)')
axes[0].set_ylabel('Y (pixels)')
plt.colorbar(im1, ax=axes[0], label='log₁₀(cells)', shrink=0.8)

# Drainage network overlay on DEM
axes[1].imshow(dem_filled, cmap='terrain', alpha=0.6, aspect='auto')
axes[1].imshow(np.ma.masked_where(~channel_network, channel_network), 
               cmap='Blues', alpha=0.8, aspect='auto')
axes[1].set_title(f'Drainage Network (threshold = {threshold_area_km2} km²)', 
                  fontsize=12, fontweight='bold')
axes[1].set_xlabel('X (pixels)')
axes[1].set_ylabel('Y (pixels)')

plt.tight_layout()
plt.show()

## Step 3: Calculate Stream Orders (Strahler)

**Theory:**
Stream ordering classifies river segments hierarchically:

**Strahler Method:**
- Headwater streams = order 1
- When two streams of order n meet → order n+1
- When streams of different orders meet → higher order continues

**Applications:**
- Understanding drainage basin organization
- Predicting discharge and sediment yield
- Geomorphic analysis

In [ ]:
def calculate_strahler_order(dem_filled, accum, threshold):
    """
    Calculate Strahler stream order.
    Simplified implementation for demonstration.
    """
    # Create channel network
    channels = accum > threshold
    
    # Initialize stream order array
    stream_order = np.zeros_like(dem_filled, dtype=int)
    
    # Simple approximation: use flow accumulation as proxy
    # Higher accumulation = higher order
    accum_channels = accum.copy()
    accum_channels[~channels] = 0
    
    # Define order thresholds based on accumulation percentiles
    valid_accum = accum_channels[channels]
    if len(valid_accum) > 0:
        p33 = np.percentile(valid_accum, 33)
        p66 = np.percentile(valid_accum, 66)
        
        stream_order[channels & (accum_channels <= p33)] = 1
        stream_order[channels & (accum_channels > p33) & (accum_channels <= p66)] = 2
        stream_order[channels & (accum_channels > p66)] = 3
    
    return stream_order

# Calculate stream orders
print("Calculating Strahler stream orders...")
stream_orders = calculate_strahler_order(dem_filled, accum, threshold_area_km2 / pixel_area_km2)
print("Stream order calculation complete!")

# Statistics
for order in range(1, 4):
    count = np.sum(stream_orders == order)
    if count > 0:
        print(f"Order {order}: {count} pixels ({count * 30 / 1000:.2f} km)")

In [ ]:
# Visualize stream orders
plt.figure(figsize=(12, 10))
plt.imshow(dem_filled, cmap='terrain', alpha=0.5, aspect='auto')

# Create custom colormap for stream orders
from matplotlib.colors import ListedColormap
colors = ['none', '#3498db', '#2ecc71', '#e74c3c']  # transparent, blue, green, red
cmap_orders = ListedColormap(colors)

im = plt.imshow(np.ma.masked_where(stream_orders == 0, stream_orders), 
                cmap=cmap_orders, vmin=0, vmax=3, alpha=0.9, aspect='auto')
cbar = plt.colorbar(im, ticks=[0, 1, 2, 3], label='Stream Order', shrink=0.8)
cbar.set_ticklabels(['None', '1st', '2nd', '3rd'])

plt.title('Strahler Stream Orders', fontsize=14, fontweight='bold')
plt.xlabel('X (pixels)')
plt.ylabel('Y (pixels)')
plt.tight_layout()
plt.show()

## Step 4: Generate Slope-Area Plots

**Theory:**
The slope-area relationship is fundamental to understanding landscape evolution:

$S = k_s A^{-\theta}$

Where:
- $S$ = local channel slope
- $A$ = drainage area
- $k_s$ = steepness index (reflects rock uplift/erosion rate)
- $\theta$ = concavity index (typically 0.3-0.6)

**Log-log transformation:**
$\log(S) = \log(k_s) - \theta \log(A)$

**Interpretation:**
- Slope decreases with increasing drainage area
- Deviations indicate tectonic activity, lithologic boundaries, or base-level changes

In [ ]:
# Calculate slope
print("Calculating slope...")
slope = rd.TerrainAttribute(dem_filled, attrib='slope_riserun')
print("Slope calculation complete!")

print(f"Slope range: {np.min(slope):.4f} to {np.max(slope):.4f} m/m")

In [ ]:
# Extract slope and area for channel pixels
channel_mask = channel_network & (slope > 0) & (accum_area > 0)
slope_channels = slope[channel_mask]
area_channels = accum_area[channel_mask]

# Remove outliers for better visualization
valid = (slope_channels > np.percentile(slope_channels, 1)) & \
        (slope_channels < np.percentile(slope_channels, 99))
slope_clean = slope_channels[valid]
area_clean = area_channels[valid]

# Perform log-log regression
log_area = np.log10(area_clean)
log_slope = np.log10(slope_clean)

# Linear fit: log(S) = log(ks) - theta * log(A)
coeffs = np.polyfit(log_area, log_slope, 1)
theta = -coeffs[0]  # Concavity index
log_ks = coeffs[1]  # Log of steepness index
ks = 10**log_ks

print(f"Slope-Area Analysis Results:")
print(f"Concavity index (θ): {theta:.3f}")
print(f"Steepness index (ks): {ks:.6f}")
print(f"Number of channel points analyzed: {len(area_clean)}")

In [ ]:
# Create slope-area plot
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Regular scale with density
axes[0].hexbin(area_clean, slope_clean, gridsize=50, cmap='viridis', 
               mincnt=1, alpha=0.8, edgecolors='none')
axes[0].set_xlabel('Drainage Area (km²)', fontsize=11)
axes[0].set_ylabel('Channel Slope (m/m)', fontsize=11)
axes[0].set_title('Slope-Area Relationship', fontsize=12, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Log-log scale with regression line
axes[1].hexbin(area_clean, slope_clean, gridsize=50, cmap='viridis',
               xscale='log', yscale='log', mincnt=1, alpha=0.8, edgecolors='none')

# Add regression line
area_fit = np.logspace(np.log10(area_clean.min()), np.log10(area_clean.max()), 100)
slope_fit = ks * area_fit**(-theta)
axes[1].plot(area_fit, slope_fit, 'r-', linewidth=2, 
             label=f'S = {ks:.2e} × A$^{{-{theta:.2f}}}$')

axes[1].set_xlabel('Drainage Area (km²)', fontsize=11)
axes[1].set_ylabel('Channel Slope (m/m)', fontsize=11)
axes[1].set_title('Log-Log Slope-Area Plot', fontsize=12, fontweight='bold')
axes[1].legend(fontsize=10, loc='best')
axes[1].grid(True, alpha=0.3, which='both')

plt.tight_layout()
plt.show()

## Step 5: Extract Longitudinal Profiles for 2-3 Rivers

**Theory:**
Longitudinal river profiles show elevation versus distance from source:
- **Equilibrium profile:** Smooth, concave-up curve
- **Knickpoints:** Sharp slope breaks indicating disequilibrium
- **Profile evolution:** Reflects balance between uplift and erosion

**Profile equation:**
$z(x) = z_b + k_s \cdot (\frac{A_0}{A(x)})^{\theta}$

Where $z$ = elevation, $x$ = distance, $A$ = area

In [ ]:
def extract_river_profile(dem, accum, start_point, min_accum=100):
    """
    Extract longitudinal profile by following flow from a starting point.
    
    Parameters:
    - dem: filled DEM
    - accum: flow accumulation
    - start_point: (row, col) starting location
    - min_accum: minimum accumulation to continue
    """
    profile_points = []
    current = start_point
    visited = set()
    
    # D8 flow direction: 8 neighboring cells (NW, N, NE, W, E, SW, S, SE)
    # Offsets in (row, col) format for the 8 cardinal and diagonal directions
    neighbors = [(-1,-1), (-1,0), (-1,1), (0,-1), (0,1), (1,-1), (1,0), (1,1)]
    
    while True:
        r, c = current
        
        # Check bounds and if visited
        if (r < 0 or r >= dem.shape[0] or c < 0 or c >= dem.shape[1] or 
            current in visited):
            break
        
        visited.add(current)
        
        # Record point
        elevation = dem[r, c]
        accumulation = accum[r, c]
        profile_points.append((r, c, elevation, accumulation))
        
        # Stop if accumulation too low
        if accumulation < min_accum:
            break
        
        # Find downstream neighbor (highest accumulation)
        max_accum = -1
        next_point = None
        
        for dr, dc in neighbors:
            nr, nc = r + dr, c + dc
            if (0 <= nr < dem.shape[0] and 0 <= nc < dem.shape[1] and 
                (nr, nc) not in visited):
                if accum[nr, nc] > max_accum and accum[nr, nc] > accumulation:
                    max_accum = accum[nr, nc]
                    next_point = (nr, nc)
        
        if next_point is None:
            break
        
        current = next_point
        
        # Limit profile length
        if len(profile_points) > 1000:
            break
    
    if len(profile_points) < 2:
        return None
    
    # Convert to arrays
    profile_array = np.array(profile_points)
    rows = profile_array[:, 0]
    cols = profile_array[:, 1]
    elevations = profile_array[:, 2]
    accumulations = profile_array[:, 3]
    
    # Calculate distances (assuming 30m pixels)
    distances = np.zeros(len(rows))
    for i in range(1, len(rows)):
        dr = rows[i] - rows[i-1]
        dc = cols[i] - cols[i-1]
        distances[i] = distances[i-1] + np.sqrt(dr**2 + dc**2) * 30  # meters
    
    distances_km = distances / 1000  # Convert to km
    
    return {
        'distance': distances_km,
        'elevation': elevations,
        'accumulation': accumulations,
        'rows': rows,
        'cols': cols
    }

# Find starting points (high accumulation points)
# Select 3 points with high accumulation but spatially separated
print("Finding river starting points...")
high_accum_mask = accum > np.percentile(accum[channel_network], 90)
high_accum_points = np.argwhere(high_accum_mask)

# Select 3 diverse starting points
if len(high_accum_points) >= 3:
    indices = np.linspace(0, len(high_accum_points)-1, 3, dtype=int)
    start_points = [tuple(high_accum_points[i]) for i in indices]
else:
    start_points = [tuple(p) for p in high_accum_points[:3]]

# Extract profiles
profiles = []
for i, start in enumerate(start_points):
    print(f"Extracting profile {i+1} from point {start}...")
    profile = extract_river_profile(dem_filled, accum, start, min_accum=100)
    if profile is not None:
        profiles.append(profile)
        print(f"  Profile length: {profile['distance'][-1]:.2f} km")
        print(f"  Elevation change: {profile['elevation'][0] - profile['elevation'][-1]:.2f} m")

print(f"\nExtracted {len(profiles)} river profiles")

In [ ]:
# Plot longitudinal profiles
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Profile locations on map
axes[0].imshow(dem_filled, cmap='terrain', alpha=0.7, aspect='auto')
colors = ['red', 'blue', 'green']
for i, profile in enumerate(profiles):
    axes[0].plot(profile['cols'], profile['rows'], 
                color=colors[i % 3], linewidth=2, 
                label=f'River {i+1}', alpha=0.8)
axes[0].set_title('River Profile Locations', fontsize=12, fontweight='bold')
axes[0].set_xlabel('X (pixels)')
axes[0].set_ylabel('Y (pixels)')
axes[0].legend(fontsize=10)

# Longitudinal profiles
for i, profile in enumerate(profiles):
    axes[1].plot(profile['distance'], profile['elevation'], 
                color=colors[i % 3], linewidth=2, marker='o', 
                markersize=3, label=f'River {i+1}', alpha=0.7)

axes[1].set_xlabel('Distance from Source (km)', fontsize=11)
axes[1].set_ylabel('Elevation (m)', fontsize=11)
axes[1].set_title('Longitudinal River Profiles', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3)
axes[1].legend(fontsize=10)

plt.tight_layout()
plt.show()

## Step 6: Identify Knickpoints and Classify Causes

**Theory:**
Knickpoints are sharp breaks in river profile slope that indicate:

**1. Tectonic knickpoints:**
- Fault scarps
- Differential uplift
- Stationary, steep

**2. Lithologic knickpoints:**
- Resistant rock layers
- Stationary
- Related to geology

**3. Base-level knickpoints:**
- Sea-level changes
- Lake formation/drainage
- Migrate upstream

**Detection methods:**
- Slope breaks
- Curvature analysis
- Chi-plot deviations

In [ ]:
def detect_knickpoints(profile, threshold_factor=1.5):
    """
    Detect knickpoints using slope change analysis.
    
    Parameters:
    - profile: dictionary with distance and elevation
    - threshold_factor: sensitivity for knickpoint detection
    
    Returns:
    - indices of knickpoint locations
    """
    distance = profile['distance']
    elevation = profile['elevation']
    
    if len(distance) < 5:
        return []
    
    # Calculate local slope
    local_slope = np.abs(np.gradient(elevation, distance))
    
    # Calculate second derivative (curvature)
    curvature = np.abs(np.gradient(local_slope, distance))
    
    # Smooth to reduce noise
    window = min(5, len(curvature) // 10)
    if window > 1:
        curvature_smooth = ndimage.uniform_filter1d(curvature, window)
    else:
        curvature_smooth = curvature
    
    # Find peaks in curvature
    threshold = np.mean(curvature_smooth) + threshold_factor * np.std(curvature_smooth)
    knickpoints = []
    
    for i in range(1, len(curvature_smooth) - 1):
        if (curvature_smooth[i] > threshold and 
            curvature_smooth[i] > curvature_smooth[i-1] and 
            curvature_smooth[i] > curvature_smooth[i+1]):
            knickpoints.append(i)
    
    return knickpoints

# Detect knickpoints for each profile
all_knickpoints = []
for i, profile in enumerate(profiles):
    knickpoints = detect_knickpoints(profile, threshold_factor=1.5)
    all_knickpoints.append(knickpoints)
    print(f"River {i+1}: {len(knickpoints)} knickpoints detected")
    for kp_idx in knickpoints:
        dist = profile['distance'][kp_idx]
        elev = profile['elevation'][kp_idx]
        print(f"  Knickpoint at {dist:.2f} km, elevation {elev:.1f} m")

In [ ]:
# Visualize profiles with knickpoints
fig, axes = plt.subplots(len(profiles), 1, figsize=(14, 5*len(profiles)))

if len(profiles) == 1:
    axes = [axes]

colors_rivers = ['red', 'blue', 'green']

for i, (profile, knickpoints) in enumerate(zip(profiles, all_knickpoints)):
    # Plot profile
    axes[i].plot(profile['distance'], profile['elevation'], 
                color=colors_rivers[i % 3], linewidth=2, 
                marker='o', markersize=3, label='River profile', alpha=0.7)
    
    # Mark knickpoints
    if len(knickpoints) > 0:
        kp_dist = profile['distance'][knickpoints]
        kp_elev = profile['elevation'][knickpoints]
        axes[i].scatter(kp_dist, kp_elev, color='red', s=200, 
                       marker='*', edgecolors='black', linewidths=1.5,
                       label='Knickpoints', zorder=5)
    
    axes[i].set_xlabel('Distance from Source (km)', fontsize=11)
    axes[i].set_ylabel('Elevation (m)', fontsize=11)
    axes[i].set_title(f'River {i+1} - Longitudinal Profile with Knickpoints', 
                     fontsize=12, fontweight='bold')
    axes[i].grid(True, alpha=0.3)
    axes[i].legend(fontsize=10)

plt.tight_layout()
plt.show()

### Knickpoint Classification Guidelines

To classify the causes of detected knickpoints, consider:

**1. Morphological characteristics:**
- **Slope**: Very steep = likely tectonic/lithologic; Moderate = possible base-level
- **Shape**: Angular = tectonic; Rounded = erosional or base-level
- **Height**: Large vertical drop = major control (fault, resistant lithology)

**2. Spatial patterns:**
- **Alignment**: Multiple rivers at same elevation = base-level change
- **Stationary**: Same location across tributaries = lithologic or tectonic
- **Progressive**: Upstream migration = base-level wave

**3. External data (if available):**
- Geologic maps (lithology)
- Fault maps (tectonics)
- Sea-level/lake history (base-level)

**Exercise:** For each detected knickpoint, examine its characteristics and propose a likely cause.

## Step 7: Exercise - Compare Concavity Index (θ) for Tectonically Active vs. Stable Basins

**Background:**
The concavity index (θ) reflects the balance between erosion processes:

**Typical values:**
- **Tectonically active basins**: θ ≈ 0.4-0.6 (steeper, more convex profiles)
- **Stable basins**: θ ≈ 0.3-0.5 (gentler, more concave profiles)
- **Detachment-limited**: θ ≈ 0.5
- **Transport-limited**: θ ≈ 1.0

**Factors affecting θ:**
- Uplift rate
- Rock erodibility
- Climate (precipitation)
- Sediment supply

**Task:**
Compare θ values between different rivers or regions to infer tectonic activity.

In [ ]:
def calculate_profile_concavity(profile):
    """
    Calculate concavity index for a single river profile.
    Uses slope-area relationship: S = ks * A^(-theta)
    """
    distance = profile['distance']
    elevation = profile['elevation']
    area = profile['accumulation'] * (30 * 30 / 1e6)  # Convert to km²
    
    # Calculate local slope
    slope = np.abs(np.gradient(elevation, distance * 1000))  # m/m
    
    # Remove invalid values
    valid = (slope > 0) & (area > 0) & np.isfinite(slope) & np.isfinite(area)
    slope_valid = slope[valid]
    area_valid = area[valid]
    
    if len(slope_valid) < 10:
        return None, None
    
    # Log-log regression
    log_area = np.log10(area_valid)
    log_slope = np.log10(slope_valid)
    
    # Remove outliers
    q1_s, q99_s = np.percentile(log_slope, [1, 99])
    q1_a, q99_a = np.percentile(log_area, [1, 99])
    outlier_mask = ((log_slope > q1_s) & (log_slope < q99_s) & 
                   (log_area > q1_a) & (log_area < q99_a))
    
    if np.sum(outlier_mask) < 10:
        return None, None
    
    log_area_clean = log_area[outlier_mask]
    log_slope_clean = log_slope[outlier_mask]
    
    # Fit: log(S) = log(ks) - theta * log(A)
    coeffs = np.polyfit(log_area_clean, log_slope_clean, 1)
    theta = -coeffs[0]
    ks = 10**coeffs[1]
    
    return theta, ks

# Calculate concavity for each river
print("\n=== Concavity Index Analysis ===")
print("\nRiver-specific concavity indices:")
concavities = []

for i, profile in enumerate(profiles):
    theta, ks = calculate_profile_concavity(profile)
    if theta is not None:
        concavities.append(theta)
        print(f"River {i+1}:")
        print(f"  Concavity index (θ) = {theta:.3f}")
        print(f"  Steepness index (ks) = {ks:.6f}")
        
        # Interpret
        if theta > 0.5:
            regime = "transport-limited / tectonically stable"
        elif theta > 0.35:
            regime = "detachment-limited / moderately active"
        else:
            regime = "highly active tectonic setting"
        print(f"  Interpretation: {regime}")

if len(concavities) > 0:
    print(f"\nMean concavity index: {np.mean(concavities):.3f} ± {np.std(concavities):.3f}")

In [ ]:
# Create comparison visualization
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Concavity comparison
if len(concavities) > 0:
    river_names = [f'River {i+1}' for i in range(len(concavities))]
    colors_bar = ['red', 'blue', 'green'][:len(concavities)]
    
    bars = axes[0].bar(river_names, concavities, color=colors_bar, alpha=0.7, edgecolor='black')
    axes[0].axhline(y=0.5, color='orange', linestyle='--', linewidth=2, 
                    label='Typical detachment-limited (θ=0.5)')
    axes[0].axhline(y=0.35, color='red', linestyle='--', linewidth=2, 
                    label='Active tectonic threshold (θ=0.35)')
    axes[0].set_ylabel('Concavity Index (θ)', fontsize=11)
    axes[0].set_title('Concavity Index Comparison', fontsize=12, fontweight='bold')
    axes[0].legend(fontsize=9)
    axes[0].grid(True, alpha=0.3, axis='y')
    axes[0].set_ylim(0, max(concavities) * 1.2 if max(concavities) > 0 else 1)

# Plot 2: Reference comparison with literature
reference_data = {
    'Himalaya\n(active)': 0.42,
    'Appalachians\n(stable)': 0.55,
    'Taiwan\n(very active)': 0.38,
    'Amazon\n(cratonic)': 0.65,
    'This Study\n(mean)': np.mean(concavities) if len(concavities) > 0 else 0
}

regions = list(reference_data.keys())
theta_values = list(reference_data.values())
colors_ref = ['tomato', 'lightblue', 'orangered', 'lightgreen', 'gold']

bars2 = axes[1].bar(regions, theta_values, color=colors_ref, alpha=0.7, edgecolor='black')
axes[1].axhline(y=0.5, color='gray', linestyle='--', linewidth=1, alpha=0.5)
axes[1].set_ylabel('Concavity Index (θ)', fontsize=11)
axes[1].set_title('Comparison with Global Reference Basins', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='y')
axes[1].tick_params(axis='x', rotation=0)

plt.tight_layout()
plt.show()

### Discussion Questions

Based on your analysis, consider:

1. **What do the concavity values suggest about tectonic activity in your study area?**
   - θ < 0.4: Potentially active tectonics, rapid uplift
   - θ = 0.4-0.6: Moderate activity, balanced conditions
   - θ > 0.6: Stable, cratonic conditions

2. **How consistent are θ values between different rivers?**
   - High consistency: Regional control (climate, lithology, tectonics)
   - High variability: Local controls, different erosional regimes

3. **How do your values compare with reference basins?**
   - Similar to active orogens (Himalaya, Taiwan): Suggest active tectonics
   - Similar to stable regions (Appalachians, Amazon): Suggest stability

4. **What other factors could affect concavity?**
   - Lithologic variations
   - Climate gradients
   - Glacial history
   - Base-level changes

5. **How do knickpoints relate to concavity patterns?**
   - Knickpoints may represent transient adjustment
   - Profile segments between knickpoints may have different θ values

## Conclusion and Summary

In this lab, you have:

1. ✓ **Preprocessed DEM data** using pit-filling algorithms
2. ✓ **Extracted drainage networks** using flow accumulation and threshold area
3. ✓ **Calculated stream orders** using the Strahler method
4. ✓ **Generated slope-area plots** and derived concavity/steepness indices
5. ✓ **Extracted longitudinal river profiles** for multiple rivers
6. ✓ **Identified knickpoints** and discussed their possible causes
7. ✓ **Compared concavity indices** to interpret tectonic activity

---

### Key Takeaways

- **River profiles** record landscape evolution and tectonic history
- **Slope-area relationships** reveal erosion processes and uplift rates
- **Concavity index (θ)** is a diagnostic tool for tectonic activity
- **Knickpoints** mark disequilibrium and landscape transience
- **Drainage metrics** provide quantitative constraints on geomorphic processes

---

### Further Exploration

To extend this analysis:

1. **Use real DEM data** from your study area (SRTM, ASTER, LiDAR)
2. **Perform chi (χ) analysis** for more robust profile comparison
3. **Integrate with geologic/tectonic maps** for knickpoint classification
4. **Compare multiple basins** across tectonic boundaries
5. **Model landscape evolution** using numerical models (e.g., Landlab, FastScape)
6. **Analyze temporal changes** using multi-temporal DEMs

---

### References

**Key Papers:**
- Barnes, R., et al. (2014). "Priority-flood: An optimal depression-filling algorithm." *Computers & Geosciences*.
- Perron, J.T., & Royden, L. (2013). "An integral approach to bedrock river profile analysis." *Earth Surface Processes and Landforms*.
- Wobus, C., et al. (2006). "Tectonics from topography: Procedures, promise, and pitfalls." *GSA Special Papers*.
- Whipple, K.X., & Tucker, G.E. (1999). "Dynamics of the stream-power river incision model." *JGR: Solid Earth*.

**Software:**
- RichDEM: https://github.com/r-barnes/richdem
- TopoToolbox: https://topotoolbox.wordpress.com/
- LSDTopoTools: https://lsdtopotools.github.io/

---

**End of Lab 3**